In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
from finrl.meta.preprocessor.preprocessors import data_split
from environements.portfolio_optimization_env import PortfolioOptimizationEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from utils.feature_engineer import FeatureEngineer
from models import DRLAgent
import sys

sys.path.append("../FinRL-Library")

from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

from pandas import read_csv

def add_volatility(df, periods=21):
    rolling_volatility = df.groupby('tic')['log_return'].rolling(window=periods).std()
    rolling_volatility = rolling_volatility.reset_index(level=0, drop=True)
    # Assign the annualized volatility back to the original DataFrame
    df['volatility'] = rolling_volatility
    
    # Fill missing values with 0 (first periods will have NaN)
    df['volatility'].fillna(0, inplace=True)
    
    return df
    


df_dow =read_csv('./data/dow.csv')
df_nasdaq =read_csv('./data/nasdaq.csv')
df_hsi = read_csv('./data/hsi.csv')
df_dax = read_csv('./data/dax.csv')
df_sp500 = read_csv('./data/sp500.csv')

In [2]:
df =df_dow.copy()

In [5]:
columns = reduced_data.columns.tolist()
df = df_dow[df_dow.tic.isin(columns)]

In [6]:
TRAIN_START_DATE = '2014-01-01'
TRAIN_END_DATE = '2019-12-30'

VALIDATION_START_DATE = '2020-01-01'
VALIDATION_END_DATE = '2020-12-30'

TEST_START_DATE = '2021-01-01'
TEST_END_DATE = '2024-10-01'
INDICATORS = [
    "close_21_ema",
    "close_62_ema"
    ]

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=False,
                     user_defined_feature = True)

processed_dax = fe.preprocess_data(df.query('date>"2013-01-01"'))
cleaned_data = processed_dax.copy()
cleaned_data = cleaned_data.fillna(0)
cleaned_data = cleaned_data.replace(np.inf,0)
train_data = data_split(cleaned_data, TRAIN_START_DATE, TRAIN_END_DATE)
test_data = data_split(cleaned_data, TEST_START_DATE, TEST_END_DATE)
validation_data = data_split(cleaned_data, VALIDATION_START_DATE, VALIDATION_END_DATE)
stock_dimension = len(train_data.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}")


Successfully added technical indicators
Successfully added user defined features
Stock Dimension: 7


In [4]:
pv = train_data.pivot(columns='tic',values='close')
corr = pv.corr()
data = corr.copy()
high_corr_pairs = set()
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.9:
            col_name = corr.columns[i]
            high_corr_pairs.add(col_name)

# Step 3: Drop one of the correlated columns
reduced_data = data.drop(columns=high_corr_pairs)
print("Original columns:", data.columns.tolist())
print("Columns retained after filtering:", reduced_data.columns.tolist())
reduced_data = reduced_data.drop(high_corr_pairs)
reduced_data.style.background_gradient(cmap='coolwarm')

Original columns: ['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']
Columns retained after filtering: ['AAPL', 'AXP', 'DIS', 'GS', 'IBM', 'MMM', 'WBA']


tic,AAPL,AXP,DIS,GS,IBM,MMM,WBA
tic,,,,,,,
AAPL,1.000000,0.828584,0.847921,0.693695,-0.106555,0.683521,-0.293444
AXP,0.828584,1.000000,0.680693,0.517039,0.108471,0.449335,-0.624728
DIS,0.847921,0.680693,1.000000,0.520113,-0.200584,0.463319,-0.135335
GS,0.693695,0.517039,0.520113,1.000000,0.209985,0.784357,0.009485
IBM,-0.106555,0.108471,-0.200584,0.209985,1.000000,-0.045360,-0.260052
MMM,0.683521,0.449335,0.463319,0.784357,-0.045360,1.000000,0.052103
WBA,-0.293444,-0.624728,-0.135335,0.009485,-0.260052,0.052103,1.000000


In [7]:
cleaned_data = add_volatility(cleaned_data)
train_data = data_split(cleaned_data, TRAIN_START_DATE, TRAIN_END_DATE)
test_data = data_split(cleaned_data, TEST_START_DATE, TEST_END_DATE)
validation_data = data_split(cleaned_data, VALIDATION_START_DATE, VALIDATION_END_DATE)
stock_dimension = len(train_data.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}")

Stock Dimension: 7


In [8]:
from utils.portfolio_trainer import PortfolioOptimization


optimizer = PortfolioOptimization()
optimizer.train_model( train_data,
        test_data,
        validation_data,
        policy_network="MultiInputPolicy",
        model_name="ppo")

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: You must use `MultiInputPolicy` when working with dict observation space, not MlpPolicy